In [381]:
#%matplotlib inline               
import pandas as pd              
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
np.random.seed(1337)  # for reproducibility
#import matplotlib.pyplot as plt

import scipy
import scipy.io.wavfile
#import IPython
from sklearn.cluster import KMeans
import os 
import glob
#from python_speech_features import delta
#from python_speech_features import logfbank
import scipy.io.wavfile as wav
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


import random
#from keras.datasets import mnist
#from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers
from keras import  regularizers
from keras.layers import LSTM

from sklearn.metrics import confusion_matrix
#from sklearn.model_selection import train_test_split


In [382]:
#batch_size = 25
#nb_classes = 14
#nb_epoch = 15

# input image dimensions
#img_rows, img_cols = 40, 25
# number of convolutional filters to use
#nb_filters = 32
# size of pooling area for max pooling
#pool_size = (3, 3)
# convolution kernel size
#kernel_size = (5, 3)

In [455]:
train_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_02Jan_labels_ids.csv')
#train_afds
#train_lre = pd.read_csv('/home/satishk/ivectors_csv_revised/train_feat_BNF_h5_07Nov_Shreyas.csv')

In [456]:
train_lre.shape

(114276, 504)

In [457]:
#train_lre = train_lre.iloc[1000:2000]

In [458]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,494,495,496,497,498,499,ids,year,data,lang
0,-0.639420,0.345687,-0.517642,-0.736995,1.313009,1.655726,0.615175,0.799342,1.648420,1.314984,...,-2.216082,-1.322349,-0.716935,-3.843975,-1.471226,0.945404,zkllk,LDC2017E22,data,spa-car
1,0.113609,0.738035,0.584858,-0.248514,0.256618,1.060176,-0.416205,0.133669,0.188325,0.805241,...,-0.082178,1.965844,3.525489,0.006793,1.676278,-1.116285,lid05e1_lid00562,LDC2017E22,data,ara-apc
2,2.061305,-0.140602,-0.627102,-0.682902,1.337618,1.261542,-0.651286,0.307042,-0.980716,0.335533,...,0.996222,-3.018732,-1.841219,0.814874,-0.200840,1.180867,fla_0240-a,LDC2017E22,data,spa-car
3,1.958050,-0.608313,-0.113530,0.167277,0.911105,1.116085,-0.847208,-0.041355,-0.542331,0.199113,...,1.487050,-0.549122,-1.254462,1.644782,0.555329,1.159080,fla_0240-a,LDC2017E22,data,spa-car
4,1.842435,-0.417594,-0.427426,-0.485892,1.204338,1.439185,-1.056640,0.111603,-0.404314,-0.307069,...,0.712544,0.296827,-1.858668,0.209770,0.583769,1.398776,fla_0240-a,LDC2017E22,data,spa-car


In [459]:
train_lre.groupby(['lang']).size()

lang
ara-acm    10560
ara-apc    23683
ara-ary     6002
ara-arz     2775
eng-gbr      720
eng-usg    17482
por-brz     3293
qsl-pol     3851
qsl-rus     7890
spa-car     4462
spa-eur      692
spa-lac     6048
zho-cmn    26241
zho-nan      577
dtype: int64

In [460]:
#train_lre=train_lre.drop(train_lre.columns[0], axis=1)


In [461]:
#train_lre

In [462]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_02Jan.csv')
#val_afds

In [463]:
#val_lre = val_lre.iloc[100:300]

In [464]:
val_lre.shape

(3661, 505)

In [465]:
val_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,uttid,language_code,data_source,speech_duration,segmentid1
0,-0.888894,-0.725546,1.228264,2.474096,-0.136334,1.062179,-0.099016,-0.470705,1.654057,-1.702968,...,0.208034,0.467388,-0.336385,1.096538,-1.589270,lre17_jxjmxcks,spa-lac,mls14,30,lre17_jxjmxcks
1,-0.387867,-0.468535,-1.576731,0.049272,-0.716639,0.822751,0.408834,-1.029209,2.657421,0.756408,...,-1.132769,-0.464028,0.401513,-0.841627,1.031664,lre17_iaxztiph,zho-cmn,vast,1000,lre17_iaxztiph
2,-0.167892,-0.524584,-1.522212,-0.576468,1.031362,0.791171,2.272489,-1.107504,2.928773,2.028390,...,-1.956216,0.023479,-0.005307,1.021076,-0.648201,lre17_gzuiblur,por-brz,vast,1000,lre17_gzuiblur
3,-0.195398,-0.390383,-0.624266,1.535441,0.388126,1.727447,0.463118,2.059773,0.165201,0.619945,...,0.616359,-2.654541,0.482435,0.401479,-1.639324,lre17_zpuysjrb,zho-nan,mls14,10,lre17_zpuysjrb
4,0.192936,-1.009773,0.330688,2.114603,-1.960716,1.176021,0.320783,-0.202096,0.561380,-0.318037,...,0.680556,-1.863441,-2.006974,1.963618,-0.744876,lre17_amnlzmbw,qsl-pol,mls14,3,lre17_amnlzmbw


In [466]:
X_train = train_lre.drop(['ids','year','data','lang'],axis=1)
#X_train = train_lre.drop(['langid'],axis=1)

y_train = train_lre["lang"]

#y_train_uttid = train_lre["uttid"]

In [467]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.639420,0.345687,-0.517642,-0.736995,1.313009,1.655726,0.615175,0.799342,1.648420,1.314984,...,-0.677938,1.489974,-0.093175,-0.034224,-2.216082,-1.322349,-0.716935,-3.843975,-1.471226,0.945404
1,0.113609,0.738035,0.584858,-0.248514,0.256618,1.060176,-0.416205,0.133669,0.188325,0.805241,...,0.097766,0.518305,0.966576,-0.167231,-0.082178,1.965844,3.525489,0.006793,1.676278,-1.116285
2,2.061305,-0.140602,-0.627102,-0.682902,1.337618,1.261542,-0.651286,0.307042,-0.980716,0.335533,...,0.904280,1.346115,-1.534558,-0.781364,0.996222,-3.018732,-1.841219,0.814874,-0.200840,1.180867
3,1.958050,-0.608313,-0.113530,0.167277,0.911105,1.116085,-0.847208,-0.041355,-0.542331,0.199113,...,0.358685,1.931928,1.998663,0.149433,1.487050,-0.549122,-1.254462,1.644782,0.555329,1.159080
4,1.842435,-0.417594,-0.427426,-0.485892,1.204338,1.439185,-1.056640,0.111603,-0.404314,-0.307069,...,-0.424766,1.179233,0.954688,-2.461002,0.712544,0.296827,-1.858668,0.209770,0.583769,1.398776


In [468]:
y_train.head()

0    spa-car
1    ara-apc
2    spa-car
3    spa-car
4    spa-car
Name: lang, dtype: object

In [469]:
val_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,uttid,language_code,data_source,speech_duration,segmentid1
0,-0.888894,-0.725546,1.228264,2.474096,-0.136334,1.062179,-0.099016,-0.470705,1.654057,-1.702968,...,0.208034,0.467388,-0.336385,1.096538,-1.589270,lre17_jxjmxcks,spa-lac,mls14,30,lre17_jxjmxcks
1,-0.387867,-0.468535,-1.576731,0.049272,-0.716639,0.822751,0.408834,-1.029209,2.657421,0.756408,...,-1.132769,-0.464028,0.401513,-0.841627,1.031664,lre17_iaxztiph,zho-cmn,vast,1000,lre17_iaxztiph
2,-0.167892,-0.524584,-1.522212,-0.576468,1.031362,0.791171,2.272489,-1.107504,2.928773,2.028390,...,-1.956216,0.023479,-0.005307,1.021076,-0.648201,lre17_gzuiblur,por-brz,vast,1000,lre17_gzuiblur
3,-0.195398,-0.390383,-0.624266,1.535441,0.388126,1.727447,0.463118,2.059773,0.165201,0.619945,...,0.616359,-2.654541,0.482435,0.401479,-1.639324,lre17_zpuysjrb,zho-nan,mls14,10,lre17_zpuysjrb
4,0.192936,-1.009773,0.330688,2.114603,-1.960716,1.176021,0.320783,-0.202096,0.561380,-0.318037,...,0.680556,-1.863441,-2.006974,1.963618,-0.744876,lre17_amnlzmbw,qsl-pol,mls14,3,lre17_amnlzmbw


In [470]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [471]:
type(X_val)

pandas.core.frame.DataFrame

In [472]:
#XX_train = X_train.append(X_val, ignore_index=True)
#y_train = y_train.append(y_val, ignore_index=True)

In [473]:
#XX_train = pd.DataFrame(XX_train)

In [474]:
#type(XX_train)

In [475]:
#XX_train.shape

In [476]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [477]:
X_train.shape

(102848, 500)

In [478]:
X_test.shape

(11428, 500)

In [479]:
y_val.head()

0    spa-lac
1    zho-cmn
2    por-brz
3    zho-nan
4    qsl-pol
Name: language_code, dtype: object

In [480]:
y_train.head()

41085    qsl-pol
30219    ara-acm
34391    qsl-rus
47483    spa-car
77358    zho-cmn
Name: lang, dtype: object

In [481]:
y_val.shape

(3661,)

In [482]:
type(y_train)

pandas.core.series.Series

In [483]:
#y_train = y_train.append(y_val, ignore_index=True)

In [484]:
y_train.shape

(102848,)

In [485]:
yyy = y_train

In [486]:
yyy.head()

41085    qsl-pol
30219    ara-acm
34391    qsl-rus
47483    spa-car
77358    zho-cmn
Name: lang, dtype: object

In [487]:
y_train.head()

41085    qsl-pol
30219    ara-acm
34391    qsl-rus
47483    spa-car
77358    zho-cmn
Name: lang, dtype: object

In [488]:
#y_val_segmentid.head()

In [489]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [490]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [491]:
y_train=le.transform(y_train)

In [492]:
y_train.shape

(102848,)

In [493]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [494]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [495]:
y_val_labels = le.transform(y_val)

In [496]:
y_val_labels[0:10]

array([11, 12,  6, 13,  7,  0,  7,  5,  5, 12])

In [497]:
y_test = le.transform(y_test)

In [498]:
y_test

array([ 8,  0, 12, ...,  9,  1, 12])

In [499]:
X_train=X_train.values
X_val=X_val.values
X_test=X_test.values

In [500]:
X_train.shape

(102848, 500)

In [501]:
X_val.shape

(3661, 500)

In [502]:
y_train

array([ 7,  0,  8, ...,  6,  1, 12])

In [503]:
nb_classes = 14

In [504]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)

In [505]:
Y_val

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [506]:
Y_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [507]:
X_train.shape

(102848, 500)

In [508]:
X_val.shape

(3661, 500)

In [509]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
#print(XX_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (102848, 500)
11428 test samples
3661 val samples


In [510]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [511]:
Y_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [512]:
y_train.shape

(102848,)

In [513]:
X_train.shape

(102848, 500)

In [514]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [515]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               256512    
_________________________________________________________________
activation_16 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_17 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 14)                7182      
__________

In [516]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [517]:
best_weights_filepath = '/home/satishk/best_weights_2l_MLP_combined.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [518]:
batch_size = 256
nb_epoch=20

In [519]:
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val , Y_val),callbacks=[saveBestModel])

Train on 102848 samples, validate on 3661 samples
Epoch 1/20
Epoch 00000: val_acc improved from -inf to 0.12374, saving model to /home/satishk/best_weights_2l_MLP_combined.hdf5
9s - loss: 1.8751 - acc: 0.3700 - val_loss: 3.8645 - val_acc: 0.1237
Epoch 2/20
Epoch 00001: val_acc did not improve
9s - loss: 1.5552 - acc: 0.4434 - val_loss: 4.3175 - val_acc: 0.1218
Epoch 3/20
Epoch 00002: val_acc did not improve
9s - loss: 1.4565 - acc: 0.4705 - val_loss: 4.5876 - val_acc: 0.1229
Epoch 4/20
Epoch 00003: val_acc improved from 0.12374 to 0.12838, saving model to /home/satishk/best_weights_2l_MLP_combined.hdf5
9s - loss: 1.3700 - acc: 0.5019 - val_loss: 4.9470 - val_acc: 0.1284
Epoch 5/20
Epoch 00004: val_acc improved from 0.12838 to 0.12893, saving model to /home/satishk/best_weights_2l_MLP_combined.hdf5
9s - loss: 1.2986 - acc: 0.5263 - val_loss: 4.8906 - val_acc: 0.1289
Epoch 6/20
Epoch 00005: val_acc improved from 0.12893 to 0.13002, saving model to /home/satishk/best_weights_2l_MLP_combin

In [520]:
#reload best weights
model.load_weights(best_weights_filepath)
#model.load_weights('/home/q/Desktop/e10_512_32.hdf5

In [521]:
#Frame label accuracy
score = model.evaluate(X_val, Y_val, verbose=0)
print('ERROR:', 1-score[1])

ERROR: 0.869980879531


In [ ]:
yyy = pd.DataFrame(yyy)

In [ ]:
yyy.shape

In [ ]:
aa = yyy.groupby(yyy.columns[0]).size()

In [ ]:
aa

In [ ]:
bb=aa.values

In [ ]:
lang_priors = bb/17879.0

In [ ]:
type(lang_priors)

In [ ]:
log_lang_priors = np.log(lang_priors)

In [ ]:
X_eval = pd.read_csv('/home/satishk/ivectors_csv_revised/eval_feat_BNF_h5_25451_07Nov.csv')

In [ ]:
X_eval.rename(columns={'uttid':'segmentid'}, inplace=True)

In [ ]:
#X_eval['segmentid'] = X_eval['uttid']


X_eval_f = X_eval.drop(["segmentid"],axis=1)
#y_eval = val_lre["language_code"]
#y_eval_segmentid = X_eval["segmentid"]

In [ ]:
y_val_segmentid = X_eval["segmentid"]

In [ ]:
X_eval_f = X_eval_f.values

X_eval_trans = X_eval_f.T

X_eval_wccn = X_wccn.dot(X_eval_trans)
X_eval_wccn = X_eval_wccn.T
X_eval = unit_len_norm(X_eval_wccn)

In [ ]:
X_eval = V.T.dot(X_eval.T)
X_eval = X_eval.T

In [ ]:
posteriors = classif.predict_proba(X_eval)

In [ ]:
log_posteriors = np.log(posteriors)

In [ ]:
log_likelihood = log_posteriors-log_lang_priors

In [ ]:
#log_likelihood = posteriors-log_lang_priors

In [ ]:
log_likelihood[0]

In [ ]:
type(log_likelihood)

In [ ]:
df_post = pd.DataFrame(log_likelihood)

In [ ]:
df_post.columns = ['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg', 'por-brz', 'qsl-pol',
                   'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac', 'zho-cmn','zho-nan']

In [ ]:
df_post.head()

In [ ]:
#pred_labels = model.predict_classes(X_val)

#df_labels = pd.DataFrame(pred_labels)

#df_labels

#y_val_uttid

In [ ]:
df_y_val_segmentid=  pd.DataFrame(y_val_segmentid)

In [ ]:
df_y_val_segmentid

#df_y_val_uttid.columns = ['segmentid']    


#df_y_val_uttid

In [ ]:
yy = pd.concat([df_y_val_segmentid,df_post],axis=1)



#yy.to_csv('/home/satishk/ivectors_csv/test_dev_30sec.csv',index=False)

#yy['segmentid'] = yy['segmentid'].astype(str) + '.sph'

#yy.to_csv('/home/satishk/ivectors_csv/test_dev_01oct.csv',mode = 'a',index=False,header=False)

In [ ]:
yy.to_csv('/home/satishk/ivectors_csv_new/eval_Nov_SVM_Shreyas.csv',index=False)

In [ ]:
test_dev_scr = pd.read_csv('/home/satishk/ivectors_csv_new/eval_Nov_SVM_Shreyas.csv')

In [ ]:
test_dev_scr = test_dev_scr.drop("segmentid.1",axis=1)

In [ ]:
test_dev_scr.to_csv('/home/satishk/Desktop/system_output_15Nov/eval_SVM_Nov_25451.scr', header=True, index=False, sep='\t')